## Notebook Description
- Cohort descriptives

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils_prediction.database import *
c = gbq_connect()

Google Big Query Connection Established


##### Query Builder

In [2]:
def build_cohort_query(cohort_table):
    return f"""
select
    c.subject_id,
    c.group_var,
    c.label,
    p.gender,
    p.anchor_age as age,
    a.ethnicity,
    a.marital_status,
    a.insurance,
from 
    `{cohort_table}` c
    left join `mimic-iv-ches.core.patients` p on c.subject_id=p.subject_id
    left join `mimic-iv-ches.core.admissions` a on c.hadm_id=a.hadm_id

"""

##### Grab Cohorts

In [3]:
dfs = {
    'Mortality': gbq_query(c, build_cohort_query('mimic-iv-ches.cohorts.mimic4ds_inhospmort'), verbose=False),
    'Long LOS': gbq_query(c, build_cohort_query('mimic-iv-ches.cohorts.mimic4ds_longlos'), verbose=False),
    'Invasive Ventilation': gbq_query(c, build_cohort_query('mimic-iv-ches.cohorts.mimic4ds_vent'), verbose=False),
    'Sepsis': gbq_query(c, build_cohort_query('mimic-iv-ches.cohorts.mimic4ds_sepsis3'), verbose=False)
}

In [7]:
df_table = pd.DataFrame()

for key in dfs.keys():
    # replace strings
    dfs[key]['gender'].replace({'F':1,'M':0},inplace=True)
    dfs[key]['ethnicity_mapped'] = np.where(dfs[key]['ethnicity']=='WHITE',1,0)
    
    # aggregate
    df = dfs[key].groupby('group_var').agg(
        subject_count = ('subject_id','count'),
        perc_pos_labels = ('label','mean'), 
        age_mean = ('age','mean'),
        age_std = ('age','std'),
        gender_F = ('gender','sum'),
        ethnicity_white = ('ethnicity_mapped','sum')
    ).reset_index()
    
    df['gender_M'] = df['subject_count'] - df['gender_F']
    df['gender_F_perc'] = df['gender_F']/df['subject_count']*100
    df['gender_M_perc'] = df['gender_M']/df['subject_count']*100
    
    df['ethnicity_Other'] = df['subject_count'] - df['ethnicity_white']
    df['ethnicity_Other_perc'] = df['ethnicity_Other']/df['subject_count']*100
    df['ethnicity_white_perc'] = df['ethnicity_white']/df['subject_count']*100
    
    # combine columns & percentages
    df['perc_pos_labels'] = df['perc_pos_labels']*100
    df['subject_count'] = df['subject_count'].apply('{:.0f}'.format)+' ('+df['perc_pos_labels'].apply('{:.1f}'.format)+'%)'
    df['gender_M'] = df['gender_M'].apply('{:.0f}'.format)+' ('+df['gender_M_perc'].apply('{:.0f}'.format)+'%)'
    df['gender_F'] = df['gender_F'].apply('{:.0f}'.format)+' ('+df['gender_F_perc'].apply('{:.0f}'.format)+'%)'
    df['ethnicity_Other'] = df['ethnicity_Other'].apply('{:.0f}'.format)+' ('+df['ethnicity_Other_perc'].apply('{:.0f}'.format)+'%)'
    df['ethnicity_white'] = df['ethnicity_white'].apply('{:.0f}'.format)+' ('+df['ethnicity_white_perc'].apply('{:.0f}'.format)+'%)'
    
    # combine age mean & std
    df['age'] = df['age_mean'].apply('{:.0f}'.format)+'±'+df['age_std'].apply('{:.0f}'.format)+''
    
    # combine
    df['Task'] = key
    df = df.rename(
        columns={
            'group_var':'Year Group',
            'subject_count':'Sample Size',
            'age':'Age',
            'gender_F':'Gender (Female)',
            'gender_M':'Gender (Male)',
            'ethnicity_white':'Ethnicity (White)',
            'ethnicity_Other':'Ethnicity (Non-white)',
        }
    )
    
    # format 
    df.drop(columns=['perc_pos_labels','age_mean','age_std'],inplace=True)
    df = df.melt(id_vars = ['Task','Year Group'])
    
    # combine dfs
    df_table = pd.concat((
        df_table,
        df.pivot(
            columns='Year Group',
            values=['value'],
            index=['Task','variable'])
    ))

# re-index
df_table = df_table.reindex(labels = [
    'Sample Size',
    'Age',
    'Gender (Female)',
    'Gender (Male)',
    'Ethnicity (White)', 
    'Ethnicity (Non-white)'
],level=1)

In [8]:
df_table

value                \
Year Group                                   2008 - 2010   2011 - 2013   
Task                 variable                                            
Mortality            Sample Size             9042 (7.4%)   9476 (7.1%)   
                     Age                           63±18         63±18   
                     Gender (Female)          3864 (43%)    4090 (43%)   
                     Gender (Male)            5178 (57%)    5386 (57%)   
                     Ethnicity (White)        6784 (75%)    6217 (66%)   
                     Ethnicity (Non-white)    2258 (25%)    3259 (34%)   
Long LOS             Sample Size            9042 (29.8%)  9476 (28.4%)   
                     Age                           63±18         63±18   
                     Gender (Female)          3864 (43%)    4090 (43%)   
                     Gender (Male)            5178 (57%)    5386 (57%)   
                     Ethnicity (White)        6784 (75%)    6217 (66%)   
                     Ethnicity (Non-white)    2258 (25%)    3259 (34%)   
Invasive Ventilation Sample Size            6692 (10.2%)  7181 (10.1%)   
                     Age                           64±18         64±18   
                     Gender (Female)          2947 (44%)    3133 (44%)   
                     Gender (Male)            3745 (56%)    4048 (56%)   
                     Ethnicity (White)        5078 (76%)    4839 (67%)   
                     Ethnicity (Non-white)    1614 (24%)    2342 (33%)   
Sepsis               Sample Size            5410 (12.7%)  5557 (10.2%)   
                     Age                           62±19         62±18   
                     Gender (Female)          2334 (43%)    2442 (44%)   
                     Gender (Male)            3076 (57%)    3115 (56%)   
                     Ethnicity (White)        4032 (75%)    3648 (66%)   
                     Ethnicity (Non-white)    1378 (25%)    1909 (34%)   

                                                                          
Year Group                                    2014 - 2016    2017 - 2019  
Task                 variable                                             
Mortality            Sample Size             10289 (7.4%)   10060 (7.2%)  
                     Age                            64±17          64±17  
                     Gender (Female)           4430 (43%)     4170 (41%)  
                     Gender (Male)             5859 (57%)     5890 (59%)  
                     Ethnicity (White)         6468 (63%)     6129 (61%)  
                     Ethnicity (Non-white)     3821 (37%)     3931 (39%)  
Long LOS             Sample Size            10289 (31.0%)  10060 (35.2%)  
                     Age                            64±17          64±17  
                     Gender (Female)           4430 (43%)     4170 (41%)  
                     Gender (Male)             5859 (57%)     5890 (59%)  
                     Ethnicity (White)         6468 (63%)     6129 (61%)  
                     Ethnicity (Non-white)     3821 (37%)     3931 (39%)  
Invasive Ventilation Sample Size             7447 (12.1%)   7311 (11.4%)  
                     Age                            64±17          64±17  
                     Gender (Female)           3318 (45%)     3124 (43%)  
                     Gender (Male)             4129 (55%)     4187 (57%)  
                     Ethnicity (White)         4920 (66%)     4727 (65%)  
                     Ethnicity (Non-white)     2527 (34%)     2584 (35%)  
Sepsis               Sample Size             6217 (10.8%)   7161 (10.6%)  
                     Age                            62±18          63±17  
                     Gender (Female)           2809 (45%)     2948 (41%)  
                     Gender (Male)             3408 (55%)     4213 (59%)  
                     Ethnicity (White)         3945 (63%)     4447 (62%)  
                     Ethnicity (Non-white)     2272 (37%)     2714 (38%)